# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [5]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [17]:
import pandas as pd

In [9]:
import textdistance

In [6]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [ ]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [40]:
def get_closest_hybrid_match(text, X, vec, topn=4, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_prob = {word:[distance, P(word)] for word, distance in dict(closest).items()}
    df = pd.DataFrame.from_dict(closest_prob, orient='index', columns=['distance', 'prob'])
    df = df.sort_values(by=['distance', 'prob'], ascending=[False, False]).drop_duplicates(['distance'])

    return closest, df.to_dict(orient='index')
    

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [22]:
get_closest_hybrid_match("приветб", X, vec)

[('привет', 0.8571428571428572), ('приветов', 0.75), ('привезти', 0.75)]

In [42]:
no_prob, with_prob = get_closest_hybrid_match('приветб', X, vec)

In [43]:
no_prob

[('привет', 0.8571428571428572),
 ('приветов', 0.75),
 ('привезти', 0.75),
 ('приве', 0.7142857142857143)]

In [44]:
with_prob

{'привет': {'dist': 0.8571428571428572, 'prob': 2.7156629156249353e-06},
 'привезти': {'dist': 0.75, 'prob': 1.3578314578124677e-06},
 'приве': {'dist': 0.7142857142857143, 'prob': 3.879518450892764e-07}}

In [45]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [46]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [47]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7926963481740871
0.0
0.09004249454461927


## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [51]:
WORDS = list(vocab)
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [52]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [53]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [54]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8479239619809905
0.15527950310559005
0.0496152520960147


In [58]:
correction("приветб")

'привет'

## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 